### Simulation Engine for Replenishment Strategy

#### Initialization

In [1]:
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt

#### "Demand" Parameters


In [2]:
C = 10      # customers per day
N = 10      # NOOS bar code in assortment
F = 5       # FASHION bar code in assortment
P = N + F   # total products in assortment 
D = 40      # days of simulation

#### "Inventory" Parameters

In [3]:
z = 1.64                # Index linked to a desired service level (95%)
LT_N = 2                # Lead time for NOOS products (days)
LT_F = 4                # Lead time for FASHION products (days)

K = 10                  # Fixed ordering cost
h = 0.44                # Holding cost per day per item
delay = 5               # Days of delay in returning frozen inventory    
limit = 7               # Capacity of the box sent to the customer 

I = np.zeros((D+LT_F+delay,P))      # Inventory level  
I[0,:] += 20
pipeline = np.zeros((D+LT_F+delay,P))   # Inventory in the pipeline (from customer to warehouse)
arrivals = np.zeros((D+LT_F+delay,P))   # Inventory in the pipeline (from supplier to warehouse)

t1 = 0.03               # Delivery fee = 3% * order cost
t2 = 0.06               # Delivery fee = 6% * order cost
t3 = 0.1                # Delivery fee = 10% * order cost

#### "Accounting" Parameters 

In [4]:
day_tot_cost = 0                                                   # Cost of sold goods per day
payment_term_soft = 5                                              # Delay in the payment - soft consignment
payment_term_norm = 1                                              # Delay in the payment - normal consignment 
account_payables = np.zeros(D+payment_term_soft)                   # Table of due payments

day_tot_rev = 0                                                    # Revenues of the day
revenues = np.zeros(D+payment_term_soft)                           # Timeline of cash inflows

#### Setup of Budgets, Prices, Costs

In [5]:
NOOS_price = np.random.uniform(30,110, N)          # NOOS products prices

cost_N = np.random.uniform(30,80, N)               # NOOS products total cost
ord_cost_N = cost_N.copy()                         # NOOS order cost
cost_N[:3] += t1 * cost_N[:3]
cost_N[3:7] += t2 * cost_N[3:7]                    
cost_N[7:10] += t3 * cost_N[7:10]                  # Delivery fees on NOOS products
                                                              
FASHION_price = np.random.uniform(160,210, F)      # FASHION products prices

cost_F = np.random.uniform(80,160, F)              # FASHION products total cost
ord_cost_F = cost_F.copy()                         # FASHION order cost
cost_F[0:2] += t1 * cost_F[:2]
cost_F[2:4] += t2 * cost_F[2:4]
cost_F[4:5] += t3 * cost_F[4:5]                    # Delivery fees on FASHION products

prob = np.array([0.03, 0.05, 0.07, 0.1, 0.15, 0.2, 0.4])    # Purchase probabilities - the higher the preference, the higher the probability

#### Setup of NOOS and FASHION variables

In [6]:
day_profit_N = 0                                                 # Daily profit from NOOS products
list_NOOS_profit = np.zeros(D)                                   # List of all daily NOOS profits
profits_per_item_N = np.zeros((N), dtype=float)                  # Profits per NOOS items
daily_soldq_N = np.zeros((D,N))                                  # List of daily sold NOOS items
avg_q_N = np.zeros((D,N))                                                    # average qt sold per day per product - NOOS
sigma_q_N = np.zeros((D,N))                                                  # std dev of qt sold per day per product - NOOS
ROP_N = np.zeros((D,N))                                                      # Reorder point for NOOS
EOQs_N = np.zeros((D,N))                                                     # Reorder quantity for NOOS
volume_N = 0                                                                 # Total volume sold 



day_profit_F = 0                                                 # Daily profit from FASHION products
list_FASHION_profit = np.zeros(D)                                # List of daily FASHION profit
profits_per_item_F = np.zeros((F), dtype=float)                  # Profits per FASHION items
daily_soldq_F = np.zeros((D,F))                                  # List of daily sold FASHION items
avg_q_F = np.zeros((D,F))                                                    # average qt sold per day per product - FASHION
sigma_q_F = np.zeros((D,F))                                                  # std dev of qt sold per day per product - FASHION
ROP_F = np.zeros((D,F))                                                      # Reorder point for FASHION
EOQs_F = np.zeros((D,F))                                                     # Reorder quantity fo FASHION
volume_F = 0                                                                 # Total volume sold

considered = 10                                           # Subset considered by each customer
lost_sales = 0                                            # Lost sales (not spent residual budget) because out of stock - Quantity
lost_sales_euro = 0                                       # Lost sales (not spent residual budget) because out of stock - Euro


served_1 = 0                                              # How many times we served the customer with the most favourite item
served_2 = 0
served_3 = 0 
served_4 = 0
nw_served = 0                                             # Not well served 
not_served = 0                                            # Not served at all


purchased_items = np.zeros((D,C))                         # How many items does each customer buy 
budg_track = np.zeros((D,C))                              # Keeping track of the budgets each customer has available

#### Model

In [ ]:
for ix_d in range(D):
    preferences = np.random.rand(P, C) 
    budgets = np.random.uniform(250, 1000, C)
    day_profit_N = 0
    day_profit_F = 0
    day_tot_cost = 0
    day_tot_rev = 0
    print("Day",ix_d)
    for ix_c in range(C):
        box = 0 
        fake_bdg = np.copy(budgets[ix_c])
        budg_track[ix_d,ix_c] += budgets[ix_c]
        ranked = np.argsort(preferences[:,ix_c])
        ranked_preferences = ranked[::-1]  
        PLS = np.zeros(considered)                                                                   # Potential Lost sales
        rk = 0                                                                                       # Counter to rank Lost Sales
        service_check = np.zeros(limit)                                                              # Service quality
        sc = 0                                                                                       # Counter for service quality
        print("Budget customer",ix_c+1, ": ", budgets[ix_c])
        for ix_rp in ranked_preferences[0:considered]:
            if ix_rp < N:
                print("  ", str(ix_rp) + ".", preferences[ix_rp,ix_c], NOOS_price[ix_rp], budgets[ix_c])
                if I[ix_d,ix_rp] > 0 and box < limit:                                                      # Box fill 
                    service_check[sc] += ix_rp
                    sc += 1
                    dec = np.random.choice([0,1] , 1, p=[prob[box], 1-prob[box]])                          # Purchase Decision
                    box += 1
                    if dec == 1 and NOOS_price[ix_rp] < budgets[ix_c]:
                        budgets[ix_c] -= NOOS_price[ix_rp]   
                        fake_bdg -= NOOS_price[ix_rp] 
                        print(" ✓✓")
                        volume_N += 1
                        day_tot_rev += NOOS_price[ix_rp]
                        day_profit_N += (NOOS_price[ix_rp] - cost_N[ix_rp])
                        profits_per_item_N[ix_rp] +=  NOOS_price[ix_rp] - cost_N[ix_rp]
                        day_tot_cost += cost_N[ix_rp] 
                        daily_soldq_N[ix_d,ix_rp] += 1
                        I[ix_d,ix_rp] -= 1
                        purchased_items[ix_d,ix_c] += 1
                    else:                                              
                        I[ix_d,ix_rp] -= 1
                        pipeline[ix_d,ix_rp] += 1                                                          # Returned items
                        print("xx")
                elif I[ix_d,ix_rp] == 0 and box <= limit and NOOS_price[ix_rp] < fake_bdg:                 # Fake budget is a copy of the normal budget to later check for lost sales
                    PLS[rk] += NOOS_price[ix_rp]
                    rk += 1
                    print("**")
                else:
                    print("//")
                avg_q_N[ix_d,ix_rp] = np.mean(daily_soldq_N[:ix_d,ix_rp])                                  # Average Demand
                sigma_q_N[ix_d,ix_rp] = np.std(daily_soldq_N[:ix_d,ix_rp])                                 # Standard deviation
                if ix_d == 0:
                    ROP_N[ix_d,ix_rp] = (LT_N * avg_q_N[ix_d,ix_rp]) + (z * sigma_q_N[ix_d,ix_rp] * math.sqrt(LT_N))
                    EOQs_N[ix_d,ix_rp] = math.sqrt((2 * K * avg_q_N[ix_d,ix_rp]) / h)
                else:                                                                                                      # ROP and EOQ
                    ROP_N[ix_d,ix_rp] = int((LT_N * avg_q_N[ix_d,ix_rp]) + (z * sigma_q_N[ix_d,ix_rp] * math.sqrt(LT_N)))
                    EOQs_N[ix_d,ix_rp] = int(math.sqrt((2 * K * avg_q_N[ix_d,ix_rp]) / h))         
            else:
                ixF = ix_rp-N
                print("  ", str(ix_rp) + ".", preferences[ix_rp,ix_c], FASHION_price[ixF], budgets[ix_c])    
                if I[ix_d,ix_rp] > 0 and box < limit:
                    service_check[sc] += ix_rp
                    sc += 1
                    dec = np.random.choice([0,1] , 1, p=[prob[box], 1-prob[box]])
                    box += 1      
                    if dec == 1 and FASHION_price[ixF] < budgets[ix_c]:
                        budgets[ix_c] -= FASHION_price[ixF]   
                        fake_bdg -= FASHION_price[ixF] 
                        print(" ✓✓")
                        volume_F += 1
                        day_tot_rev += FASHION_price[ixF]
                        day_profit_F += (FASHION_price[ixF] - cost_F[ixF])
                        profits_per_item_F[ixF] +=  FASHION_price[ixF] - cost_F[ixF]
                        day_tot_cost += cost_F[ixF] 
                        daily_soldq_F[ix_d,ixF] += 1
                        I[ix_d,ix_rp] -= 1
                        purchased_items[ix_d,ix_c] += 1
                    else:                                              
                        I[ix_d,ix_rp] -= 1
                        pipeline[ix_d,ix_rp] += 1
                        print("xx")
                elif I[ix_d,ix_rp] == 0 and box <= limit and FASHION_price[ixF] < fake_bdg:
                    PLS[rk] += FASHION_price[ixF]
                    rk += 1
                    print("**")
                else:
                    print("//")
                avg_q_F[ix_d,ixF] = np.mean(daily_soldq_F[:ix_d,ixF])
                sigma_q_F[ix_d,ixF] = np.std(daily_soldq_F[:ix_d,ixF])
                if ix_d == 0:
                    ROP_F[ix_d,ixF] = (LT_F * avg_q_F[ix_d,ixF]) + (z * sigma_q_F[ix_d,ixF] * math.sqrt(LT_F))
                    EOQs_F[ix_d,ixF] = math.sqrt((2 * K * avg_q_F[ix_d,ixF]) / h)
                else:
                    ROP_F[ix_d,ixF] = int((LT_F * avg_q_F[ix_d,ixF]) + (z * sigma_q_F[ix_d,ixF] * math.sqrt(LT_F)))
                    EOQs_F[ix_d,ixF] = int(math.sqrt((2 * K * avg_q_F[ix_d,ixF]) / h)) 
        
        for ix_sc in service_check[service_check!=0]:                                                                 #Service quality control
            if ix_sc == ranked_preferences[0]:
                served_1 += 1
                break
            elif ix_sc == ranked_preferences[1]:
                served_2 += 1
                break
            elif ix_sc == ranked_preferences[2]:
                served_3 += 1
                break
            elif ix_sc == ranked_preferences[3]:
                served_4 += 1
                break
            else:
                nw_served += 1
                break
                
        if box == 0:
            not_served += 1
        
        for ix_bp in PLS[PLS!=0]:                                                                             # Lost sales control
            if ix_bp < fake_bdg:
                lost_sales += 1
                lost_sales_euro += ix_bp
                fake_bdg -= ix_bp

    for ix_pn in range(P):                                                                         # Replenishment system
        if ix_pn < N:
            if ix_d-delay >= 0:
                I[ix_d,ix_pn] += pipeline[ix_d-delay,ix_pn]
            if ix_d-LT_N >= 0:
                I[ix_d,ix_pn] += arrivals[ix_d-LT_N,ix_pn]
            I[ix_d+1,ix_pn] = I[ix_d,ix_pn] 
            if I[ix_d,ix_pn] + np.sum(pipeline[ix_d:ix_d+delay,ix_pn]) + np.sum(arrivals[ix_d:ix_d+LT_N,ix_pn]) <= ROP_N[ix_d,ix_pn]:
                arrivals[ix_d+LT_N,ix_pn] += EOQs_N[ix_d,ix_pn]  
                day_tot_cost += K
        else:
            ix_pf = ix_pn-N
            if ix_d-delay >= 0:
                I[ix_d,ix_pn] += pipeline[ix_d-delay,ix_pn]
            if ix_d-LT_F >= 0:
                I[ix_d,ix_pn] += arrivals[ix_d-LT_F,ix_pn]
            I[ix_d+1,ix_pn] = I[ix_d,ix_pn] 
            if I[ix_d,ix_pn] + np.sum(pipeline[ix_d:ix_d+delay,ix_pn]) + np.sum(arrivals[ix_d:ix_d+LT_F,ix_pn]) <= ROP_F[ix_d,ix_pf]:
                arrivals[ix_d+LT_F,ix_pn] += EOQs_F[ix_d,ix_pf]
                day_tot_cost += K
        day_tot_cost += h * I[ix_d,ix_pn]
    
    revenues[ix_d] += day_tot_rev                                                              # Some descriptives
    account_payables[payment_term_soft + ix_d] += day_tot_cost * 0.5
    account_payables[payment_term_norm + ix_d] += day_tot_cost * 0.5
    print("NOOS DAY PROFIT :", day_profit_N)
    list_NOOS_profit[ix_d] += day_profit_N 
    print("FASHION DAY PROFIT :", day_profit_F)        
    list_FASHION_profit[ix_d] += day_profit_F
    

#### NOOS Results

In [8]:
NOOS_total_profit = np.sum(list_NOOS_profit)                                             # NOOS monthly profit
print("NOOS TOTAL PROFIT :", NOOS_total_profit)
print("PROFITS PER ITEM - NOOS: ",profits_per_item_N)
ranked_N = np.argsort(profits_per_item_N)
ranked_1 = ranked_N[::-1]                                                    # Sorted NOOS products by performance
print("TOP PERFORMING NOOS PRODUCTS: ", ranked_1)

print("Inventory Level - NOOS: ", I[ix_d,:N])
print("Reorder point - NOOS: ", ROP_N[ix_d,:])
print("Order quantity - NOOS", EOQs_N[ix_d,:])

#### FASHION Results

In [9]:
FASHION_total_profit = np.sum(list_FASHION_profit)                                           # FASHION monthly profit
print("FASHION TOTAL PROFIT :", FASHION_total_profit)
print("PROFITS PER ITEM - FASHION: ",profits_per_item_F)
ranked_F = np.argsort(profits_per_item_F)
ranked_2 = ranked_F[::-1]                                                  # Sorted FASHION products by performance
print("TOP PERFORMING FASHION PRODUCTS: ", ranked_2)

print("Inventory Level - FASHION: ", I[ix_d,N:P])
print("Reorder point - FASHION: ", ROP_F[ix_d,:])
print("Order quantity - FASHION", EOQs_F[ix_d,:])

#### General results

In [10]:
tot_lost_sales_ratio = (lost_sales / (volume_N + volume_F))
print("LOST SALES RATIO: ", tot_lost_sales_ratio)

service_quality_ratio = np.sum(served_1) / (C * D)
print("SERVICE QUALITY: ", service_quality_ratio)

sold_quantities_N = np.sum(daily_soldq_N, axis = 0)
sold_quantities_F = np.sum(daily_soldq_F, axis = 0)
print("BEST SELLING PRODUCTS (sold units): ", sold_quantities_N, sold_quantities_F)
print("MOST PROFITABLE PRODUCTS: ", ranked_1, ranked_2)

#### Accounting Report

In [11]:
#print("CHECKLIST OF DUE PAYMENTS", account_payables)
#print("CASH INFLOWS", revenues)
profits_allocation = revenues - account_payables
print("PROFIT ALLOCATION", profits_allocation)

#### Charts

In [12]:
# I transition for NOOS products - line chart

plt.figure(figsize=(6, 3), layout='constrained')
av_inv_lev = np.zeros(D)
for inv in range(D):
    av_inv_lev[inv] += np.mean(I[inv,:N])

plt.plot(av_inv_lev, markersize = 3, color='black', linewidth=0.2)
plt.axhline(y=np.nanmean(ROP_N), linestyle='--', linewidth=0.9, color='g', label="ROP")
plt.axhline(y=np.nanmean(EOQs_N), linestyle='--', linewidth=0.9, color="orange", label="EOQ")
plt.axhline(y=np.nanmean(I[:,:N]), linestyle='--', linewidth=0.9, color="r", label="INV")
plt.axhline(y=np.std(I[:,:N]), linestyle='--', linewidth=0.9, color="yellow", label="STD")
plt.title("Average Inventory Level among NOOS Products")
plt.xlabel("DAYS")
plt.ylabel("Inventory level")
plt.xticks(np.arange(0,D,5))
plt.style.use("fast")

plt.legend(loc=1)
plt.show()

In [13]:
# Potential market - pie chart

slices = np.array([volume_N+volume_F, lost_sales])
labels = ["Actual Sales", "Lost Sales"]
plt.figure(figsize=(6, 4), layout='constrained')
explode = [0,0.12]


plt.pie(slices, labels=labels, wedgeprops={"edgecolor":"black"}, explode=explode,
        shadow=True, autopct=lambda p: '{:.2f}%({:.0f})'.format(p,(p/100)*slices.sum()))
plt.title("Potential Sales")


#plt.savefig("plot3.png")
plt.style.use("fast")

plt.tight_layout()
plt.show()

In [14]:
# Service quality - pie chart
pieces = np.array([served_1, served_2, served_3, served_4, nw_served, not_served])
tags = ["1st pref", "2nd pref", "3rd pref", "4th pref", "else", "not served"]
plt.figure(figsize=(6, 4), layout='constrained')
explode = [0,0,0.4,0.4,0.4,0.4]

plt.pie(pieces, labels=tags,colors=["#4CAF50","orange","coral","yellow","deepskyblue"],
        wedgeprops={"edgecolor":"black"}, explode=explode, shadow=True, autopct="%1.1f%%")
plt.title("Service Quality")
plt.style.use("fast")
#plt.savefig("plot4.png")
plt.tight_layout()
plt.show()

# Table

data_service = {'Class':["1st pref", "2nd pref", "3rd pref", "4th pref", "else", "not served"], 
                'Number of Customers':[served_1, served_2, served_3, served_4, nw_served, not_served], 
                'Percentages':[(served_1/(C*D)), (served_2/(C*D)), (served_3/(C*D)), (served_4/(C*D)), 
                      (nw_served/(C*D)), (not_served/(C*D))]}

df = pd.DataFrame(data_service)
df.style
display(df)

In [15]:
# Budget-Purchased items relationship - Scatter plot
x = (budg_track.flatten())
y = (purchased_items.flatten())

m, b = np.polyfit(x, y, 1)
plt.scatter(x, y, s=1, c="y", marker=".", edgecolor="black", linewidth=3, alpha=0.9)
plt.plot(x, m*x+b, linewidth=0.7, c="r")
plt.xlabel("Available Budget")
plt.ylabel("Number of purchased items")

plt.yticks(np.arange(8))
plt.title("Budget-Purchased items relationship")
plt.style.use("fast")
plt.grid(True)
plt.show()


#creare un annotazione con indici tipo: se uno ha in media 300 di budget è probabile che compri 1.5 prodotti e così via

#import pandas as pd

#data = {'budget':[np.arange(0,1000,100)], 'Purchased Goods':[]}
#io = pd.crosstab(x,y)
#print(io)

#### Consistency Check

In [16]:
print("1. ", avg_q_N[:,:5])
print("2. ", daily_soldq_N[:,:5])
print("3. ", I[:D,:N])
print("4. ", ROP_N[:,:5])
print("5. ", EOQs_N[:,:5])
print("6. ", pipeline[:,:5])

In [17]:
#print("1. ", avg_q_F[:,:5])
#print("2. ", daily_soldq_F[:200,:5])
#print("3. ", I[:,N:N+F])
#print("4. ", ROP_F[:,:5])
#print("5. ", EOQs_F[:,:5])
#print("6. ", pipeline[:,N:N+F])